In [0]:
import pandas as pd
import numpy as np
import json
import re
from bs4 import BeautifulSoup
import torch
import nltk
import ssl
from nltk.corpus import stopwords
import random
import spacy
import en_core_web_sm

nlp = en_core_web_sm.load()

In [0]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('stopwords')

In [0]:
sample_submission = pd.read_csv('tensorflow2-question-answering/sample_submission.csv')
#simplifiednqtrain = pd.read_json('tensorflow2-question-answering/simplified-nq-train.jsonl', orient='records', lines=True, dtype={'example_id':np.dtype('object')})
simplifiednqtest = pd.read_json('tensorflow2-question-answering/simplified-nq-test.jsonl', orient='records', lines=True, dtype={'example_id':np.dtype('object')})



In [0]:
simplifiednqtest.head()

,example_id,question_text,document_text,long_answer_candidates
0,-1220107454853145579,who is the south african high commissioner in ...,"High Commission of South Africa , London - wik...","[{'end_token': 136, 'start_token': 18, 'top_le..."
1,8777415633185303067,the office episode when they sing to michael,Michael 's Last Dundies - wikipedia <H1> Micha...,"[{'end_token': 190, 'start_token': 23, 'top_le..."
2,4640548859154538040,what is the main idea of the cross of gold speech,Cross of gold speech - wikipedia <H1> Cross of...,"[{'end_token': 165, 'start_token': 12, 'top_le..."
3,-5316095317154496261,when was i want to sing in opera written,Wilkie Bard - wikipedia <H1> Wilkie Bard </H1>...,"[{'end_token': 105, 'start_token': 8, 'top_lev..."
4,-8752372642178983917,who does the voices in ice age collision course,Ice Age : Collision Course - Wikipedia <H1> Ic...,"[{'end_token': 287, 'start_token': 16, 'top_le..."


In [0]:
json_train_head = []
N_HEAD = 307373

with open('tensorflow2-question-answering/simplified-nq-train.jsonl', 'rt') as f:
    for i in range(N_HEAD):
        json_train_head.append(json.loads(f.readline()))

In [0]:
W2V_PATH = 'fastText/crawl-300d-2M.vec'

In [0]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.asarray(tokens[1:], dtype=np.float32)
    return data

In [0]:
vectors = load_vectors(W2V_PATH)

In [0]:
simplifiednqtrain = pd.DataFrame(json_train_head)

In [0]:
simplifiednqtrain.head()

,document_text,long_answer_candidates,question_text,annotations,document_url,example_id
0,Email marketing - Wikipedia <H1> Email marketi...,"[{'start_token': 14, 'top_level': True, 'end_t...",which is the most common use of opt-in e-mail ...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Em...,5655493461695504401
1,The Mother ( How I Met Your Mother ) - wikiped...,"[{'start_token': 28, 'top_level': True, 'end_t...",how i.met your mother who is the mother,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Th...,5328212470870865242
2,Human fertilization - wikipedia <H1> Human fer...,"[{'start_token': 14, 'top_level': True, 'end_t...",what type of fertilisation takes place in humans,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Hu...,4435104480114867852
3,List of National Football League career quarte...,"[{'start_token': 28, 'top_level': True, 'end_t...",who had the most wins in the nfl,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Li...,5289242154789678439
4,Roanoke Colony - wikipedia <H1> Roanoke Colony...,"[{'start_token': 32, 'top_level': True, 'end_t...",what happened to the lost settlement of roanoke,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Ro...,5489863933082811018


In [0]:
del simplifiednqtrain, train_
import gc
gc.collect()

In [0]:
train_ = simplifiednqtrain[simplifiednqtrain['annotations'].apply(lambda x: x[0]['long_answer']['start_token']!=-1)]

In [0]:
train_ = train_.reset_index(drop=True)

In [0]:
train_.head()

,document_text,long_answer_candidates,question_text,annotations,document_url,example_id
0,Email marketing - Wikipedia <H1> Email marketi...,"[{'start_token': 14, 'top_level': True, 'end_t...",which is the most common use of opt-in e-mail ...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Em...,5655493461695504401
1,The Mother ( How I Met Your Mother ) - wikiped...,"[{'start_token': 28, 'top_level': True, 'end_t...",how i.met your mother who is the mother,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Th...,5328212470870865242
2,Human fertilization - wikipedia <H1> Human fer...,"[{'start_token': 14, 'top_level': True, 'end_t...",what type of fertilisation takes place in humans,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Hu...,4435104480114867852
3,List of National Football League career quarte...,"[{'start_token': 28, 'top_level': True, 'end_t...",who had the most wins in the nfl,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Li...,5289242154789678439
4,Pom Klementieff - wikipedia <H1> Pom Klementie...,"[{'start_token': 14, 'top_level': True, 'end_t...",who played mantis guardians of the galaxy 2,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Po...,-2500044561429484630


In [0]:
html_tags = ['<P>', '</P>', '<Table>', '</Table>', '<Tr>', '</Tr>', '<Ul>', '<Ol>', '<Dl>', '</Ul>', '</Ol>', \
             '</Dl>', '<Li>', '<Dd>', '<Dt>', '</Li>', '</Dd>', '</Dt>', '</S>', '<S>',  '<H1>', '</H1>', '</Td>', '<Td>']
r_buf = ['is', 'are', 'do', 'does', 'did', 'was', 'were', 'will', 'can', 'the', 'a', 'an', 'of', 'in', 'and', 'on', \
         'what', 'where', 'when', 'which']


def clean(x, stop_words=[]):
    x = x.lower()
    TAG_RE = re.compile(r'<[^>]+>')
    x = TAG_RE.sub('', x)
    return x.strip()

In [0]:
msk = np.random.rand(len(train_)) < 0.95

In [0]:
train_data = train_[msk][0:50000]

In [0]:
validate_data = train_[~msk][0:1000]

In [0]:
def create_train_dataset(df = train_data):
    example_id = []
    document_text = []
    answer_candidate = []
    target = []
    question_text = []
    level_type = []
    for i in df[['document_text', 'long_answer_candidates', 'question_text','annotations', 'example_id']].values:
        text_split = i[0]
        example_id.append(i[-1])
        text_split_cl = clean(text_split)
        document_text.append(text_split_cl)
        question_text.append(i[2].strip())
        index_start_true = i[3][0]['long_answer']['start_token']
        index_end_true = i[3][0]['long_answer']['end_token']
        
        text_splited = text_split.split(' ')
        ansC = clean(' '.join(text_splited[index_start_true : index_end_true]))
        answer_candidate.append(ansC)
        target.append(1)
        num_pos = [[k, x] for k, x in enumerate(i[1]) 
                   if x['start_token'] == index_start_true and x['end_token'] == index_end_true][0]
        num = num_pos[0]
        level_type.append(int(num_pos[1]['top_level']))
        
        for j in range(0, 5):
            choice_ind = [x for x in range(0, len(i[1])-1) if x!=num]
            if choice_ind:
                example_id.append(i[-1])
                document_text.append(text_split_cl)
                question_text.append(i[2].strip())
                k = random.choice(choice_ind)
                index_start = i[1][k]['start_token']
                index_end = i[1][k]['end_token']
                        
                strr = clean(' '.join(text_splited[index_start : index_end]))
                answer_candidate.append(strr)
                level_type.append(int(i[1][k]['top_level']))
                target.append(0)
              
    train = pd.DataFrame()
    train['example_id'] = example_id
    train['target'] = target
    train['answer_candidate'] = answer_candidate
    train['document_text'] = document_text
    train['question_text'] = question_text
    train['level_type'] = level_type
    
    return train

In [0]:
def create_test_dataset(df=validate_data):
    example_id = []
    document_text = []
    answer_candidate = []
    target = []
    question_text = []
    level_type = []
    for i in df[['document_text', 'long_answer_candidates', 'question_text','annotations', 'example_id']].values:
        text_split = i[0]
        text_splited = text_split.split(' ')
        example_id.append(i[-1])
        text_split_cl = clean(text_split)
        document_text.append(text_split_cl)
        question_text.append(i[2].strip())
        index_start_true = i[3][0]['long_answer']['start_token']
        index_end_true = i[3][0]['long_answer']['end_token']
        ansC = clean(' '.join(text_splited[index_start_true : index_end_true]))
        answer_candidate.append(ansC)
        target.append(1)
        num_pos = [[k, x] for k, x in enumerate(i[1]) 
                   if x['start_token'] == index_start_true and x['end_token'] == index_end_true][0]
        level_type.append(int(num_pos[1]['top_level']))
        
        for j in i[1]:
            index_start = j['start_token']
            index_end = j['end_token']
            if index_start!=index_start_true and index_end!=index_end_true:
                example_id.append(i[-1])
                document_text.append(text_split_cl)
                question_text.append(i[2].strip())
                strr = clean(' '.join(text_splited[index_start : index_end]))
                answer_candidate.append(strr)
                level_type.append(int(j['top_level']))
                target.append(0)
               
    test = pd.DataFrame()
    test['example_id'] = example_id
    test['target'] = target
    test['answer_candidate'] = answer_candidate
    test['document_text'] = document_text
    test['question_text'] = question_text
    test['level_type'] = level_type
    
    return test
    

In [0]:
train_data = create_train_dataset(train_data)
validate_data = create_test_dataset(validate_data)

In [0]:
train_data.shape, validate_data.shape

((299630, 6), (131937, 6))

In [0]:
def preprocess(raw_text):
    return raw_text.lower().split(' ')

In [0]:
def get_embeddings(s1):
    vector_1 = [vectors[word] if (word and word in vectors.keys()) else np.zeros(300,) for word in preprocess(s1)]
    return vector_1

In [0]:
def get_embeddings_avg(s1):
    vector_1 = get_embeddings(s1)
    vector_1_mean = np.mean(vector_1, axis=0)
    return vector_1_mean

In [0]:
from scipy.spatial import distance
    
def cos_sim(s1, s2):
    cosine = distance.cosine(s1, s2)
    return cosine

In [0]:
def get_jaccard_sim(str1, str2):
    a = set(str1)  # .split())
    b = set(str2)  # .split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [0]:
Q_TYPE = {'what was': 0, 'what is': 1, 'what': 2, 'in what': 3, 'in which': 4, 'in': 5,
          'when': 6, 'where': 7, 'who': 8, 'why': 9, 'which': 10, 'is': 11, 'other': 12}


def get_question_type_features(sample):
    vec = np.zeros(len(Q_TYPE))
    qwords = sample.split(' ')
    other = True
    if qwords[0].lower() in Q_TYPE:
        vec[Q_TYPE[qwords[0].lower()]] = 1
        other = False
    if ' '.join(list(map(lambda x: x.lower(), qwords[0:2]))) in Q_TYPE:
        vec[Q_TYPE[' '.join(list(map(lambda x: x.lower(), qwords[0:2])))]] = 1
        other = False
    if other:
        vec[Q_TYPE['other']] = 1
    return vec

In [0]:
NER_DICT = {'gpe': 0, 'person': 1, 'org': 2, 'money': 3, 'percent': 4, 'date': 5, 'time': 6, 
            'quantity': 7, 'cardinal': 8, 'ordinal': 9, 'fac':10, 'norp':11, 'loc':12, 'product':13, 'event':14, 
           'work_of_art':15, 'law':16, 'language':17}


def get_ner_features(span_ner):
    vec = np.zeros(len(NER_DICT))
    for ner in span_ner:
        vec[NER_DICT[ner.lower()]] = 1
    return vec

In [0]:
POS_DICT = {'NNP': 0, 'JJ': 1, 'NN': 2, 'IN': 3, ',': 4, 'CC': 5, 'DT': 6, 'VBG': 7, 'VB': 8, 'NNS': 9, 'POS': 10,
            'VBZ': 11, 'RB': 12, 'TO': 13, 'FW': 14, 'PRP$': 15, 'CD': 16, 'VBN': 17, 'NNPS': 18, 'JJR': 19, 'VBP': 20,
            ':': 21, 'VBD': 22, 'PRP': 23, '#': 24, 'JJS': 25, '$': 26, 'WRB': 27, '-LRB-': 28, '-RRB-': 29, '.': 30,
            '``': 31, "''": 32, 'PDT': 33, 'MD': 34, 'WP': 35, 'RP': 36, 'WDT': 37, 'EX': 38, 'UH': 39, 'SYM': 40,
            'LS': 41, 'RBS': 42, 'RBR': 43, 'WP$': 44, 'ADD':45, 'AFX':46,'GW':47, 'HYPH':48, 'NFR':49, 'NIL':50, 'SP':51,
            'XX':52, '_SP':53, 'NFP':54}


def get_pos_features(span_pos):
    vec = np.zeros(len(POS_DICT))
    for pos in span_pos:
        vec[POS_DICT[pos]] = 1
    return vec


In [0]:
%%time
a_q_SIM = []
doc_q_SIM = []
len_a = []
len_q = []
q_type = []
ents_vec = []
pos_vec = []

for i in train_data[['answer_candidate', 'question_text', 'document_text', 'target']].values:
    if i[3] == 1:
        qq_type = get_question_type_features(i[2])
        len_qq = len(i[1].split(' '))
    en_a = nlp(i[0])
    all_ent = [ent.label_.lower() for ent in en_a.ents]
    all_pos = [ent.tag_ for ent in en_a]
    ner_f = get_ner_features(all_ent)
    ents_vec.append(ner_f)
    pos_f = get_pos_features(all_pos)
    pos_vec.append(pos_f)
    aq_f = get_jaccard_sim(i[0], i[1])
    dq_f = get_jaccard_sim(i[2], i[1])
    a_q_SIM.append(aq_f)
    doc_q_SIM.append(dq_f)
    len_a.append(len(i[0].split(' ')))
    len_q.append(len_qq)
    q_type.append(qq_type)

CPU times: user 1h 1min 51s, sys: 18.6 s, total: 1h 2min 10s
Wall time: 1h 2min 19s


In [0]:
max_len_a = max(len_a)
max_len_q = max(len_q)

len_a = [x/max_len_a for x in len_a]
len_q = [x/max_len_q for x in len_q]

In [0]:
len(doc_q_SIM)

299630

In [0]:
doc_q_SIM = np.asarray(doc_q_SIM).reshape(299630, 1)
a_q_SIM = np.asarray(a_q_SIM).reshape(299630, 1)
len_a = np.asarray(len_a).reshape(299630, 1)
len_q = np.asarray(len_q).reshape(299630, 1)
q_type = np.asarray(q_type)
ents_vec = np.asarray(ents_vec)
pos_vec = np.asarray(pos_vec)

In [0]:
len_q[0]

array([0.43478261])

In [0]:
input_features = np.hstack((a_q_SIM, doc_q_SIM,len_a,len_q,pos_vec,ents_vec,q_type))
targets = np.asarray(train_data['target'].values.tolist())
targets = targets.reshape(-1, 1)

In [0]:
from numpy import zeros, newaxis
input_features = input_features[:, :, newaxis]

In [0]:
%%time
a_q_SIM_test = []
doc_q_SIM_test = []
len_a_test = []
len_q_test = []
q_type_test = []
ents_vec_test = []
pos_vec_test = []

for i in validate_data[['answer_candidate', 'question_text', 'document_text', 'target']].values:
    if i[3] == 1:
        qq_type = get_question_type_features(i[2])
        len_qq = len(i[1].split(' '))
    en_a = nlp(i[0])
    all_ent = [ent.label_.lower() for ent in en_a.ents]
    all_pos = [ent.tag_ for ent in en_a]
    ner_f = get_ner_features(all_ent)
    ents_vec_test.append(ner_f)
    pos_f = get_pos_features(all_pos)
    pos_vec_test.append(pos_f)
    aq_f = get_jaccard_sim(i[0], i[1])
    dq_f = get_jaccard_sim(i[2], i[1])
    a_q_SIM_test.append(aq_f)
    doc_q_SIM_test.append(dq_f)
    len_a_test.append(len(i[0].split(' ')))
    len_q_test.append(len_qq)
    q_type_test.append(qq_type)

CPU times: user 19min 46s, sys: 2.15 s, total: 19min 48s
Wall time: 19min 49s


In [0]:
max_len_a_test = max(len_a_test)
max_len_q_test = max(len_q_test)

len_a_test = [x/max_len_a_test for x in len_a_test]
len_q_test = [x/max_len_q_test for x in len_q_test]

In [0]:
doc_q_SIM_test = np.asarray(doc_q_SIM_test).reshape(131937, 1)
a_q_SIM_test = np.asarray(a_q_SIM_test).reshape(131937, 1)
len_a_test = np.asarray(len_a_test).reshape(131937, 1)
len_q_test = np.asarray(len_q_test).reshape(131937, 1)
q_type_test = np.asarray(q_type_test)
ents_vec_test = np.asarray(ents_vec_test)
pos_vec_test = np.asarray(pos_vec_test)

In [0]:
input_features_test = np.hstack((doc_q_SIM_test, a_q_SIM_test, len_a_test, len_q_test, q_type_test, ents_vec_test, pos_vec_test))
targets_test = np.asarray(validate_data['target'].values.tolist())
targets_test = targets_test.reshape(-1, 1)


In [0]:
from numpy import zeros, newaxis
input_features_test = input_features_test[:, :, newaxis]

In [0]:
input_features.shape

(299630, 90, 1)

In [0]:
targets.shape

(299630, 1)

In [0]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [0]:
import keras as K
from keras.layers import Dense, Input, Dropout, GRU
from keras.models import Sequential
from keras.optimizers import Adam

In [0]:
del model

In [0]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(90,)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [0]:
#model.save('my_model.h5')

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_3 (GRU)                  (None, 512)               789504    
_________________________________________________________________
dropout_21 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 513       
Total params: 790,017
Trainable params: 790,017
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0005), metrics=['accuracy'])

In [0]:
history = model.fit(input_features, targets, epochs=20, batch_size=256, verbose=1)

In [0]:
y_pred = model.predict(input_features_test)

In [0]:
y_pred = y_pred.reshape(1, -1)


In [0]:
targets_test.shape

(299630, 1)

In [0]:
qq = pd.DataFrame()
qq['y_pred'] = y_pred[0]
qq['example_id'] = validate_data['example_id']
qq['targets'] = targets_test

In [0]:
ans_qq = []


In [0]:
idx = qq.groupby(['example_id'])['y_pred'].transform(max) == qq['y_pred']

In [0]:
idx = idx.values.astype(int)

In [0]:
idx = idx.tolist()

In [0]:
from sklearn.metrics import f1_score, accuracy_score
f1 = f1_score(targets_test, idx)
print(f1)